In [1]:
# Import Libraries and DataSet
# Dataset is composed of RAMAN TR, RAMAN RE, NIR TR, NIR RE, Compression Force curve, Compression max, and the Dissolution
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np 
import pylab
import math
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=RuntimeWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
from numpy import loadtxt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

RamanTR = loadtxt(r'.\RAMANTR256.csv', delimiter=';', dtype='float64')
RamanRe = loadtxt(r'.\RAMANRE256.csv', delimiter=';', dtype='float64')
NIRRe = loadtxt(r'.\NIRRE256.csv', delimiter=';', dtype='float64')
NIRTR= loadtxt(r'.\NIRTR256.csv', delimiter=';', dtype='float64')
Diss = loadtxt(r'.\DISSOLUTION256.csv', delimiter=';', dtype='float64')
Comp = loadtxt(r'.\Compression256.csv', delimiter=',', dtype='float64')
CompMax = loadtxt(r'.\CompressionMax.csv', delimiter=';', dtype='float64')
Contents = loadtxt(r'.\Contents.csv', delimiter=',', dtype='float64')
XramanRe = RamanRe[:,0:1691]
XnirRe = NIRRe[:,0:1555]
XramanTR = RamanTR[:,0:1691]
XnirTR = NIRTR[:,0:713]
Xcomp = Comp[:,0:6036]
Contents = Contents[:,0:2]
XcompMax = np.array(CompMax[np.newaxis].T)
Diss=Diss[:,1:54]

In [2]:
print(Contents.shape)

(296, 2)


In [3]:
def calcf2(P,T,rows,samples):  
    rows=rows
    samples=samples
    count=0

    y_pred=P
    y_test=T
    ftotal=0
    for i in range(0, samples):

        f=0

        somme=0
        Xrow = y_test[i]
        Yrow = y_pred[i]
        for n in range(0,rows):
            Xcell=Xrow[n]
            Ycell=Yrow[n]
            Error=Xcell-Ycell
            Eabs=Error**2
            somme=somme+Eabs
        step1=1+(1/rows)*somme
        step2=step1**(-0.5)
        step3=step2*100
        f=50*math.log10( step3 )
        ftotal=ftotal+f
    averagef2=ftotal/samples
    return(averagef2)

In [4]:
#Data Preprocessing (Scaling)
XramanRescaler = StandardScaler()
ScaledRamanRE=XramanRescaler.fit_transform(XramanRe)

XnirRescaler = StandardScaler()
ScaledNirRE=XnirRescaler.fit_transform(XnirRe)

XramanTRscaler = StandardScaler()
ScaledRamanTR=XramanTRscaler.fit_transform(XramanTR)

XnirTRscaler = StandardScaler()
ScaledNirTR=XnirTRscaler.fit_transform(XnirTR)

Xcompscaler = StandardScaler()
ScaledComp=Xcompscaler.fit_transform(Xcomp)

ContentScaler = StandardScaler()
ContentsScaled=ContentScaler.fit_transform(Contents)
print(ContentsScaled.shape)

(296, 2)


In [5]:
# Different Inputs and Data Reduction using PCA 
#INPUT1 ---> PCAINPUT1 
#In this input, the data was scaled, merged together, scaled again, PCA was applied to keep 99% of the variances

INPUT1=np.hstack((ScaledNirTR,ScaledRamanRE,ScaledComp,ScaledNirRE,ScaledRamanTR))
INPUT1scaler = StandardScaler()
INPUT1scaler.fit(INPUT1)
ScaledINPUT1=INPUT1scaler.transform(INPUT1)
pca= PCA(0.99)
pca.fit(ScaledINPUT1)
PCAINPUT1= pca.transform(ScaledINPUT1)
print("Input1",PCAINPUT1.shape)



print("Dissolution",Diss.shape)

Input1 (296, 33)
Dissolution (296, 53)


In [7]:
withContents=np.hstack((PCAINPUT1,Contents))
print("With Contents",withContents.shape)


With Contents (296, 35)


In [8]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import make_scorer
import numpy as np

# Assuming PCAINPUT1 and Diss are your feature and target variables
trainings = 1
DissolutionPoints = 53
TestSize = 49
f = {}
average = 0

# Assuming PCAINPUT1 and Diss are your feature and target variables
X_train, X_test, y_train, y_test = train_test_split(withContents, Diss, test_size=0.165)

# Custom scoring function
def custom_metric(y_true, y_pred, dissolution_points, test_size):
    # Replace the following line with your own custom metric calculation (calcf2)
    score = calcf2(y_true, y_pred, dissolution_points, test_size)
    return score

# Make scorer for the custom metric
custom_scorer = make_scorer(custom_metric, greater_is_better=True, dissolution_points=DissolutionPoints, test_size=TestSize)

for i in range(0, trainings):
    mlp = MLPRegressor(hidden_layer_sizes=(10, 10, 10), activation='relu', solver='lbfgs', max_iter=1000)

    # Perform cross-validation
    cv_scores = cross_val_score(mlp, X_train, y_train, cv=5, scoring=custom_scorer)

    # Train the model on the entire training set
    mlp.fit(X_train, y_train)

    # Evaluate on the test set
    mpl_score = mlp.score(X_test, y_test)
    y_pred = mlp.predict(X_test)

    # Store performance metric
    f[i] = calcf2(y_test, y_pred, DissolutionPoints, TestSize)
    average += f[i]


    average_score = np.mean(cv_scores)
    std_deviation = np.std(cv_scores)
    confidence_interval = 1.96 * (std_deviation / np.sqrt(len(cv_scores)))

# Get the number of parameters
num_params = sum([p.size for p in mlp.coefs_]) + sum([p.size for p in mlp.intercepts_])

print("Number of parameters in the model:", num_params)
# Print results
print("Cross-validated scores:", cv_scores)
print("Average Score:", average_score)
print("Standard Deviation:", std_deviation)
print("95% Confidence Interval:", average_score - confidence_interval, "to", average_score + confidence_interval)


Number of parameters in the model: 1163
Cross-validated scores: [69.64891502 72.31749615 73.33285967 72.3376714  71.68473255]
Average Score: 71.86433495657961
Standard Deviation: 1.2268911791145038
95% Confidence Interval: 70.7889174222016 to 72.93975249095762
